In [53]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/notebook


In [54]:
from langchain_community.document_loaders import (
                                                  PyMuPDFLoader,
                                                  NewsURLLoader,
                                                  YoutubeLoader,
                                                  )
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

import pacmap
import numpy as np
import plotly.express as px


from src.utils import *
from src.ragqa import *

import pandas as pd
import random
pd.set_option('display.max_colwidth', None)

In [55]:
device = take_device()

Selected device: cuda


In [56]:
def remove_repetitive_patterns(text:str):
    # Define regex patterns to match repetitive sequences
    patterns = [
        r'(.\n\n)+',                      # Matches repetitive sequences of ".\n"
        r'(\\\(T\\.\\\)\\\(\\.\\\))+',   # Matches repetitive sequences of "\(T.\)\(.\)"
        r'(\w+)\1+',                      # Matches repetitive sequences of a word
        r'(\\!)+',
    ]
    
    # Process each pattern and replace it with a single instance
    for pattern in patterns:
        text = re.sub(pattern, r'\1', text)
    
    return text

In [57]:
text = "\!\!\!\!\!\!\!\!\!\!"
print(remove_repetitive_patterns(text))

\!


<>:1: DeprecationWarning: invalid escape sequence '\!'
<>:1: DeprecationWarning: invalid escape sequence '\!'
/tmp/ipykernel_622551/3198881728.py:1: DeprecationWarning: invalid escape sequence '\!'
  text = "\!\!\!\!\!\!\!\!\!\!"


In [58]:
model_config = config_parser(data_config_path = 'config/model_config.yaml')

In [59]:
embedding_model = HuggingFaceEmbeddings(
    model_name=model_config["embedding_model"],
    # multi_process=True,
    model_kwargs={"device": device},
    # encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

# KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
#     docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
# )
knowledge_vector_db = FAISS.load_local("data/vectorstores/db_faiss_4", 
                                        embedding_model, 
                                        allow_dangerous_deserialization='True',
                                        )

In [60]:
# Embed a user query in the same space
user_query = "what is ER model"
query_vector = embedding_model.embed_query(user_query)

In [61]:
docs_processed=[]
for key, val in knowledge_vector_db.index_to_docstore_id.items():
    docs_processed.append(knowledge_vector_db.docstore.search(val))

In [62]:
docs_processed[:5]

[Document(page_content='Chapter 3:\nEnhanced Entity-\nRelationship (ER) Model\nJan - 2015\n Contents\n2\n1\nIntroduction to Enhanced-ER Model\n2\nSubclases, Superclases, and Inheritance\n3\nSpecialization and Generalization\n4\nConstraints and Characteristics of Specialization \nand Generalization Hierarchies\n5\nCategories\n6\nDesign Choices, and Formal Definitions\nJan - 2015', metadata={'source': 'data/web_data/Chapter_3 - EERD.pdf', 'file_path': 'data/web_data/Chapter_3 - EERD.pdf', 'page': 1, 'total_pages': 67, 'format': 'PDF 1.7', 'title': 'Course:        Database Management Systems Credits: 3', 'author': 'PHONG VU COMPUTER', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® for Office 365', 'producer': 'Microsoft® PowerPoint® for Office 365', 'creationDate': "D:20190620145018+07'00'", 'modDate': "D:20190620145018+07'00'", 'trapped': ''}),
 Document(page_content='Contents\n3\n1\nIntroduction to Enhanced-ER Model\n2\nSubclases, Superclases, and Inheritance\n3\nSpec

In [63]:
strings = [doc.page_content for doc in docs_processed]

In [64]:
strings[:10]

['Chapter 3:\nEnhanced Entity-\nRelationship (ER) Model\nJan - 2015\n Contents\n2\n1\nIntroduction to Enhanced-ER Model\n2\nSubclases, Superclases, and Inheritance\n3\nSpecialization and Generalization\n4\nConstraints and Characteristics of Specialization \nand Generalization Hierarchies\n5\nCategories\n6\nDesign Choices, and Formal Definitions\nJan - 2015',
 'Contents\n3\n1\nIntroduction to Enhanced-ER Model\n2\nSubclases, Superclases, and Inheritance\n3\nSpecialization and Generalization\n4\nConstraints and Characteristics of Specialization \nand Generalization Hierarchies\n5\nCategories\n6\nDesign Choices, and Formal Definitions\nJan - 2015',
 'Introduction to Enhanced-ER Model \n(1)\n\uf06eEnhanced ER or Extended ER (ER) \nmodel\n\uf071Created to design more acurate database \nschemas \n\uf06eReflect the data properties and constraints more \nprecisely\n\uf071More complex requirements than traditional \naplications\n4\nJan - 2015',
 'Introduction to Enhanced-ER Model \n(2)\n\uf06eE

In [48]:

sentences = strings.copy()

# Function to get anchor and positive (anchor is random, positive is the next element)
def get_anchor_positive(sentences):
    anchor_index = random.randint(0, len(sentences) - 2)  # Ensure there is a next element
    anchor = sentences[anchor_index]
    positive = sentences[anchor_index + 1]
    return anchor, positive

# Function to get a negative sentence
def get_negative(sentences, anchor, positive):
    negative = random.choice(sentences)
    while negative == anchor or negative == positive:
        negative = random.choice(sentences)
    return negative

# Create a DataFrame with the specified columns
data = {'anchor': [], 'positive': [], 'negative': []}

# Populate the DataFrame with 300 rows of data
n =1000
for _ in range(n):
    anchor, positive = get_anchor_positive(sentences)
    negative = get_negative(sentences, anchor, positive)
    data['anchor'].append(anchor)
    data['positive'].append(positive)
    data['negative'].append(negative)

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('data/triplet_dataset.csv', index=False, escapechar="\\")

print("CSV file created successfully.")



CSV file created successfully.


In [49]:
df.head()

,anchor,positive,negative
0,".\n\n.\n\n, xk),\nwhere each Qi, i = 1, 2, .\n\n.\n\n.\n\n, k, is either the existential quan-\ntiﬁer or the universal quantiﬁer, and P (x1, .\n\n.\n\n.\n\n, xk) is a pred-\nicate involving no quantiﬁers.\n\nFor example, ∃x∀y(P (x, y) ∧\nQ(y)) is in prenex normal form, whereas ∃xP (x) ∨∀xQ(x)\nis not (because the quantiﬁers do not al ocur ﬁrst).\n\nEvery statement formed from propositional variables,\npredicates, T, and F using logical conectives and quan-\ntiﬁers is equivalent to a statement in prenex normal form.","Exercise 51 asks for a prof of this fact.\n\n∗50.\n\nPut these statements in prenex normal form.\n\n[Hint: Use\nlogical equivalence from Tables 6 and 7 in Section 1.3,\nTable 2 in Section 1.4, Example 19 in Section 1.4,\nExercises 45 and 46 in Section 1.4, and Exercises 48 and\n49.]\n\na) ∃xP (x) ∨∃xQ(x) ∨A, where A is a proposition not\ninvolving any quantiﬁers.\n\nb) ¬(∀xP (x) ∨∀xQ(x))\nc) ∃xP (x) →∃xQ(x)\n∗∗51.","∃x∃yP (x, y)\nThere is a pair x, y for which\nP(x, y) is false for every\n∃y∃xP (x, y)\nP(x, y) is true.\n\npair x, y.\nis true.\n\nThe order of the quantiﬁcation here is important, because the quantiﬁcation\n∃z∀x∀yQ(x, y, z),\nwhich is the statement\n“There is a real number z such that for al real numbers x and for al real numbers y it is\ntrue that x + y = z,”\nis false, because there is no value of z that satisﬁes the equation x + y = z for al values of x\nand y."
1,"Often, it is not easy to ﬁnd where the eror in reasoning ocurs in such mis-\ntaken profs.","328\n5 / Induction and Recursion\nTo uncover erors in profs by mathematical induction, rember that in every such prof,\nboth the basis step and the inductive step must be done corectly.\n\nNot completing the basis step\nin a suposed prof by mathematical induction can lead to mistaken profs of clearly ridiculous\nstatements such as “n = n + 1 whenever n is a positive integer.” (We leave it to the reader to\nshow that it is easy to construct a corect inductive step in an atempted prof of this statement.)","For example, an architect might make a decision to use React.js for \nfrontend development.\n\nIn this case, the architect is making a technical decision rather than an \narchitectural decision or design principle that wil help the development team make choices."
2,"15.\n\nLet the Chomp board have n\nrows and n columns.\n\nWe claim that the ﬁrst player can win the\ngame by making the ﬁrst move to leave just the top row and\nleftmost column.\n\nLet P (n) be the statement that if a player\nhas presented his oponent with a Chomp conﬁguration con-\nsisting of just n cokies in the top row and n cokies in the\nleftmost column, then he can win the game.\n\nWe wil prove\n∀nP (n) by strong induction.","We wil prove\n∀nP (n) by strong induction.\n\nWe know that P (1) is true, be-\ncause the oponent is forced to take the poisoned cokie at\nhis ﬁrst turn.\n\nFix k ≥1 and asume that P (j) is true for al\nj ≤k.\n\nWe claim that P (k + 1) is true.\n\nIt is the oponent’s\nturn to move.\n\nIf she picks the poisoned cokie, then the game\nis over and she loses.\n\nOtherwise, asume she picks the cokie\nin the top row in column j, or the cokie in the left column in\nrow j, for some j with 2 ≤j ≤k + 1.","0\n1\n1/2\n1/4\n0\n–1 per insertion\n+1 per deletion\n+2 per insertion\n–2 per deletion\n˛\nˆ\nˆ per operation\nT:num\nT:size\nT:size=2\nT:size=4\nT:size=2\nT:size=4\nT:size\nT:size=4\nAs the load factor moves away from 1=2, the potential builds up to pay for copying\nal the items ) the potential neds to increase to num by the time the load factor\nreaches either 1 or 1=4."
3,"This algorithm\ncan be used when the list has terms ocuring in order of increasing size (for instance: if the\nterms are numbers, they are listed from smalest to largest; if they are words, they are listed\nin lexicographic, or alphabetic, order).\n\nThis second searching algorithm is caled the binary\nsearch algorithm.\n\nIt proceds by comparing the element to be l

In [50]:
len(df)

1000